In [3]:
!pip3 install opacus

     |████████████████████████████████| 125 kB 26.1 MB/s            


In [18]:
import os
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

from sagemaker.pytorch import PyTorch

from opacus import PrivacyEngine

In [19]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

bucket = "responsibleai"
data_prefix = "data"
input_data_path = "s3://{}/{}".format(bucket, data_prefix)

print("region: " + region)
print("role: " + role)
print("input_data_path: " + input_data_path)

region: us-east-1
role: arn:aws:iam::247385444016:role/service-role/AmazonSageMaker-ExecutionRole-20210426T062919
input_data_path: s3://responsibleai/data


In [22]:
# prepare data
df = pd.read_csv("../data/churn.csv")
df = df.drop(["Surname", "CustomerId", "RowNumber"], axis=1)

# Grouping variable names
categorical = ["Geography", "Gender"]
target = "Exited"

# One-hot encoding of categorical variables
churn_frame = pd.get_dummies(df, prefix=categorical)

# Save target and predictors
X = churn_frame.drop(target, axis=1)
y = churn_frame["Exited"]

scaler = StandardScaler()
X_array  = scaler.fit_transform(X)

X = pd.DataFrame(X_array)
churn_train, churn_test = train_test_split (X, test_size=0.2)

churn_train.to_csv("../data/train_churn.csv", index=False, header=False)
churn_test.to_csv("../data/test_churn.csv", index=False, header=False)

In [ ]:
train_uri = S3Uploader.upload("../data/train_churn_dp.csv", "s3://{}/{}".format(bucket, data_prefix)) 
test_uri = S3Uploader.upload("../data/test_churn_dp.csv", "s3://{}/{}".format(bucket, data_prefix)) 

In [ ]:
!pygmentize 'churn.py'

In [ ]:
pytorch_estimator = PyTorch(entry_point = 'churn.py',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 100, 
                                               'batch-size': 64, 
                                               'learning-rate': 0.003})

In [ ]:
pytorch_estimator.fit({'train': 's3://{}/{}/train_churn_dp.csv".format(bucket, data_prefix)'})